<a href="https://colab.research.google.com/github/FelixMburu/CODE-CHALLENGES-AND-CHECKPOINTS/blob/main/Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scikit-learn surprise pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357230 sha256=625d9c77753c87d975a9de6458185d8db240bd3b688ae3d2759adea8d1c39882
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


Data Preparation
We'll load the MovieLens 100k dataset using the surprise library.

python


In [ ]:

# Load MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')

# Convert the dataset to a DataFrame for content-based filtering
raw_ratings = data.raw_ratings
ratings = pd.DataFrame(raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

# Load movie titles from MovieLens 100k dataset
movie_info = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep='|',
                         encoding='latin-1', header=None, usecols=[0, 1, 2, 3, 4],
                         names=['movieId', 'title', 'release_date', 'video_release_date', 'IMDb URL'])

# Drop unused columns
movie_info = movie_info[['movieId', 'title']]


movie_info.columns, ratings.columns

(Index(['movieId', 'title'], dtype='object'),
 Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object'))

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100000 non-null  object 
 1   movieId    100000 non-null  object 
 2   rating     100000 non-null  float64
 3   timestamp  100000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [ ]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  1682 non-null   int64 
 1   title    1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [ ]:
# change datatype before merging
movie_info['movieId'] = movie_info['movieId'].astype('str')

In [ ]:
# Merge ratings with movie information
ratings = pd.merge(ratings, movie_info, on='movieId')
ratings.head()

,userId,movieId,rating,timestamp,title
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


## Using functions

## Content-Based Filtering
We'll use TF-IDF to transform the movie genres into vectors and compute cosine similarity to recommend similar movies.

In [ ]:
# TF-IDF Vectorization of movie titles
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movie_info['title'])

# Calculate cosine similarity between movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations based on content
def content_based_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = movie_info[movie_info['title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movie_info['title'].iloc[movie_indices]

# Example usage
print(content_based_recommendations('Toy Story (1995)'))


1071                    Pyromaniac's Love Story, A (1995)
1343                         Story of Xinghua, The (1993)
477                        Philadelphia Story, The (1940)
547                     NeverEnding Story III, The (1994)
307                        FairyTale: A True Story (1997)
1052                                  Now and Then (1995)
1546                                     Show, The (1995)
1589                               To Have, or Not (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
598     Police Story 4: Project S (Chao ji ji hua) (1993)
Name: title, dtype: object


## Collaborative Filtering
We'll use Singular Value Decomposition (SVD) from the Surprise library to predict user ratings.

In [ ]:
# Prepare data for collaborative filtering
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Prepare data for collaborative filtering
trainset, testset = train_test_split(data, test_size=0.2)

# Build and train the SVD model
svd = SVD()
svd.fit(trainset)

# Function to get movie recommendations based on collaborative filtering
def collaborative_recommendations(user_id, num_recommendations=10):
    # Get all movie ids
    movie_ids = movie_info['movieId'].unique()

    # Predict ratings for all movies for the given user
    predictions = [svd.predict(user_id, movie_id) for movie_id in movie_ids]

    # Sort predictions based on the estimated rating
    predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Get the top N movie ids
    top_movie_ids = [pred.iid for pred in predictions[:num_recommendations]]

    # Return the top N recommended movie titles
    return movie_info[movie_info['movieId'].isin(top_movie_ids)]['title']

# Example usage
print(collaborative_recommendations(88))


11                            Usual Suspects, The (1995)
63                      Shawshank Redemption, The (1994)
113    Wallace & Gromit: The Best of Aardman Animatio...
168                           Wrong Trousers, The (1993)
177                                  12 Angry Men (1957)
317                              Schindler's List (1993)
407                                Close Shave, A (1995)
482                                    Casablanca (1942)
512                                Third Man, The (1949)
602                                   Rear Window (1954)
Name: title, dtype: object


## Hybrid Recommender Systems

In [ ]:
# Hybrid recommendation system combining content-based and collaborative filtering
def hybrid_recommendations(user_id, title, num_recommendations=10, alpha=0.5):
    # Content-based recommendations
    content_recs = content_based_recommendations(title)

    # Collaborative recommendations
    collaborative_recs = collaborative_recommendations(user_id, num_recommendations)

    # Combine recommendations
    combined_recs = pd.concat([content_recs, collaborative_recs]).drop_duplicates().head(num_recommendations)

    return combined_recs

# Example usage
print(hybrid_recommendations(66, 'Toy Story (1995)'))


1071                    Pyromaniac's Love Story, A (1995)
1343                         Story of Xinghua, The (1993)
477                        Philadelphia Story, The (1940)
547                     NeverEnding Story III, The (1994)
307                        FairyTale: A True Story (1997)
1052                                  Now and Then (1995)
1546                                     Show, The (1995)
1589                               To Have, or Not (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
598     Police Story 4: Project S (Chao ji ji hua) (1993)
Name: title, dtype: object


## SkLearn Library

### Content Based Filtering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Create a TF-IDF matrix of the movie titles
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movie_info['title'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create a Series to map movie titles to indices
indices = pd.Series(movie_info.index, index=movie_info['title']).drop_duplicates()

# Get the index of the movie that matches the title
idx = indices['Toy Story (1995)']

# Get the pairwise similarity scores of all movies with that movie
sim_scores = list(enumerate(cosine_sim[idx]))

# Sort the movies based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar movies
sim_scores = sim_scores[1:11]

# Get the movie indices
movie_indices = [i[0] for i in sim_scores]

# Print the top 10 most similar movies
print(movie_info['title'].iloc[movie_indices])


1071                    Pyromaniac's Love Story, A (1995)
1343                         Story of Xinghua, The (1993)
477                        Philadelphia Story, The (1940)
547                     NeverEnding Story III, The (1994)
307                        FairyTale: A True Story (1997)
1052                                  Now and Then (1995)
1546                                     Show, The (1995)
1589                               To Have, or Not (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
598     Police Story 4: Project S (Chao ji ji hua) (1993)
Name: title, dtype: object


In [ ]:
# Load the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# Build and train the SVD model
svd = SVD()
svd.fit(trainset)

# Evaluate the model
predictions = svd.test(testset)
accuracy.rmse(predictions)

# Make a prediction for a specific user and item
user_id = str(1)
movie_id = str(1)
pred = svd.predict(user_id, movie_id)
print(pred.est)

NameError: name 'accuracy' is not defined